In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
import torch_tensorrt

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

True

In [2]:
df = create_merged_game_data_df(['ranked','mango','public'])

In [3]:
df.columns


Index(['source', 'source_path_prefix', 'source_path_suffix', 'length',
       'num_players', 'is_teams', 'player_1_port', 'player_1_character_name',
       'player_1_type_name', 'player_1_stocks', 'player_1_costume',
       'player_1_team_value', 'player_1_ucf_shield_drop_name', 'player_1_tag',
       'player_1_display_name', 'player_2_port', 'player_2_character_name',
       'player_2_type_name', 'player_2_stocks', 'player_2_costume',
       'player_2_team_value', 'player_2_ucf_shield_drop_name', 'player_2_tag',
       'player_2_display_name', 'random_seed', 'slippi', 'stage_name',
       'is_pal', 'is_frozen_ps', 'end_method_name', 'lras_initiator',
       'conclusive', 'winning_player', 'date', 'duration', 'platform',
       'player_1_netplay_code', 'player_1_netplay_name',
       'player_2_netplay_code', 'player_2_netplay_name', 'console_name',
       'all_data_df_common_path', 'inputs_df_common_path',
       'inputs_np_common_path', 'player_1_all_data_df_sub_path',
       'player_

In [4]:
df['player_1_display_name'].value_counts()

                   99186
Platinum Player    47335
Master Player      39195
Diamond Player     29613
mang               14142
                   ...  
AV                     1
Zuppy                  1
tk                     1
DawsonTruu             1
Bink                   1
Name: player_1_display_name, Length: 257, dtype: int64

In [5]:
df['length'].sum() / (60)

41459841.06666667

In [6]:
def asses_model(model_name, y_pred, y_test, labels_order):
    print()
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_pred, y_test)

    # Print accuracy and Cohen Kappa score with explanations
    print(f'Accuracy of {model_name}: {accuracy:.4f}')
    print(f'Cohen Kappa Score of {model_name}: {kappa:.4f}')

    # Calculate the normalized predicted label count
    unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    normalized_counts_pred = {k: v / counts_test[np.where(unique_test == k)[0][0]] for k, v in zip(unique_pred, counts_pred)}
    
    # Calculate the percent the model over or under predicted the labels using the specified label order
    sorted_values = [normalized_counts_pred[k] - 1 if k in normalized_counts_pred else 0 for k in labels_order]

    # Plotting the percent the model over or under predicted the labels
    plt.figure(figsize=(2*len(labels_order), 4))
    plt.bar(labels_order, sorted_values, color=['green' if x > 0 else 'blue' for x in sorted_values])
    plt.title(f'Percent Model {model_name} Over or Under Predicted Labels')
    plt.xlabel('Labels')
    plt.ylabel('Percent Over/Under Prediction')
    
    # Center y-axis and set equal extension above and below
    max_extent = max(abs(min(sorted_values)), abs(max(sorted_values))) * 1.05
    plt.ylim(-max_extent, max_extent)
    plt.axhline(y=0, color='gray', linewidth=0.8)
    plt.show()

    # Display each confusion matrix on its own row
    for norm in [None, 'true', 'pred']:
        plt.figure(figsize=(len(labels_order)+5, len(labels_order)+5))
        ax = plt.gca()
        ConfusionMatrixDisplay.from_predictions(
            y_test, y_pred, normalize=norm, ax=ax,
            xticks_rotation='vertical', labels=labels_order
        )
        ax.title.set_text(f'{model_name} Confusion Matrix ({"Not Normalized" if norm is None else "Normalized by " + norm})')
        plt.tight_layout()
        plt.show()
        



In [7]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [8]:
# source_data = ['ranked','public']

# general_features = {
#     'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
#     'num_players': [2],
#     'conclusive': [True]
# }
# player_features = {
#     # 'netplay_code': ['MANG#0'],
#     # 'character_name': ['PICHU'],
#     # 'character_name': ['PIKACHU'],
#     # 'character_name': ['PIKACHU','PICHU'],
#     # 'character_name': ['FOX','FALCO'],
#     'character_name': ['FOX','FALCO','PIKACHU','PICHU'],
#     # 'display_name': ['Platinum Player','Master Player', 'Diamond Player']

    
# }
# opposing_player_features = {
#     # 'character_name': ['MARTH'],
#     # 'netplay_code': ['KOD#0', 'ZAIN#0']
#     'type_name': ['HUMAN']
# }
# label_info = {
#     'source': ['player'], # Can be 'general', 'player
#     # 'feature': ['netplay_code']
#     'feature': ['character_name']
# }

In [9]:
# # We classify opponent's characters on competitive stages

# source_data = ['ranked']

# general_features = {
#     'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
#     'num_players': [2],
#     'conclusive': [True]
# }
# player_features = {
#     # 'netplay_code': ['MANG#0'],
#     'character_name': ['FOX'],
#     'type_name': ['HUMAN']
    
# }
# opposing_player_features = {
#     'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
#     # 'netplay_code': ['KOD#0', 'ZAIN#0'],
#     'type_name': ['HUMAN']
# }
# label_info = {
#     'source': ['opposing_player'], # Can be 'general', 'player', 'opposing_player'
#     # 'feature': ['netplay_code']
#     'feature': ['character_name']
# }
    

In [10]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())

/workspace/slp_jaspar/autoencoder_classify/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


FOX               103069
FALCO              90719
MARTH              53728
CAPTAIN_FALCON     38006
SHEIK              27623
Name: labels, dtype: int64


In [11]:
labels_order =  dataset.number_of_segments_per_game(3600,40000)
print(2**10)
print(labels_order)
labels_order = labels_order['Label'].values


1024
            Label   Count  Shift
0             FOX  102551  15075
1           FALCO   90263  12994
2           MARTH   53538   8570
3  CAPTAIN_FALCON   37820   5406
4           SHEIK   27536   4950


In [12]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)

                           player_inputs_np_sub_path  length  num_segments  \
0  public\FALCO\c4923e7a-eb2b-4d7a-844d-0642caa17...   16592             1   
1  ranked\FALCO\34c40adf-7386-4603-9c9e-ee2f8e2ba...   16590             1   
2  ranked\FALCO\db3afa43-f3cf-4cba-94ce-105f6670b...   16569             1   
3  ranked\FALCO\e7d480eb-7a28-4741-b49e-917544a11...   16564             1   
4  public\FALCO\9f3fda37-6f5f-4db7-aeee-516bc89e8...   16540             1   

  labels  encoded_labels  
0  FALCO               1  
1  FALCO               1  
2  FALCO               1  
3  FALCO               1  
4  FALCO               1  


In [13]:
train_df, test_df = dataset.all_segments_train_test_split_dataframes(3600, proportion_of_segments=1, test_ratio = .2, val = False)
porportion = 1
train_df = train_df.sample(frac=porportion, random_state = 42)
porportion = .05
test_df = test_df.sample(frac=porportion, random_state = 42)
print(train_df.shape)
print(test_df.shape)
print(test_df.shape[0] / (train_df.shape[0] + test_df.shape[0]))
train_df.head()

(542672, 6)
(6783, 6)
0.012344960005823953


,player_inputs_np_sub_path,labels,encoded_labels,segment_start_index,segment_index,segment_length
351084,ranked\FOX\1e2a5e72-2de8-46d6-b2e9-d4a53da8227...,FOX,2,0,0,3600
215852,ranked\MARTH\52e795fc-d615-420f-8115-556fd2bf8...,MARTH,3,2605,1,3600
119316,ranked\FALCO\2b24c6dd-06a0-438b-a5bd-7b2440b13...,FALCO,1,5580,2,3600
526796,mango\SHEIK\53ccce5e-4093-4231-aed1-2ea15c93cb...,SHEIK,4,2600,1,3600
14671,ranked\FALCO\89abf765-69d7-4bc1-8d48-560773217...,FALCO,1,2540,1,3600


In [14]:
labels_unique = train_df['labels'].unique()
encoded_labels_unique = train_df['encoded_labels'].unique()
label_decoder = zip(labels_unique, encoded_labels_unique)
label_decoder = dict(zip(encoded_labels_unique, labels_unique)) 
print(label_decoder)

{2: 'FOX', 3: 'MARTH', 1: 'FALCO', 4: 'SHEIK', 0: 'CAPTAIN_FALCON'}


In [15]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,int(segment_start):int(segment_end)]
        

        segment[-5] = (segment[-5] > .5)
        
        if self.transform:
            transformed = np.zeros((9 + 5+4,3600))
            
            # First 4 rows of transformed are the analog inputs transformed to have range [0,1]
            # Shift inputs: adjusting so that positives are reduced and negatives are increased
            analog_transformed = np.copy(segment[0:4])
            analog_transformed[analog_transformed > 0] -= 0.2875 + 0.0125
            analog_transformed[analog_transformed < 0] += 0.2875 - 0.0125
            # Scale inputs to be between -.5 and .5
            analog_transformed *= .5 / .725
            # Add .5 to so final inputs are between 0 and 1
            analog_transformed += .5
            
            transformed[0:4] = analog_transformed
            # Next four rows are 1 if the corresponding analog input is 0
            transformed[4:8] += (segment[:4] == 0)
            
            
            prepend = np.expand_dims(segment[-5:, 0], axis=1)
            transitions= np.abs(np.diff(segment[-5:], axis=1, prepend=prepend))
            
            transformed[8:13] += transitions
            
            

            # Transform the Trigger so that it is 0 or 1
            
            
            # Remaining rows are button inputs
            transformed[-5:] += segment[-5:]
            
            
        
        # Convert to PyTorch tensors
        segment_tensor = torch.from_numpy(transformed).float()
        label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment_tensor, label_tensor
    
def prepare_data_loaders(train_df, test_df, batch_size, num_workers):
    # Initialize datasets
    train_dataset = TrainingDataset(train_df,True)
    # val_dataset = TrainingDataset(file_paths_val, labels_val)
    test_dataset = TrainingDataset(test_df,True)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True,persistent_workers=True, drop_last=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True, drop_last=True),
        # 'val': DataLoader(val_dataset, batch_size=2**9, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True)
    }
    return loaders



# ''' Get a batch of data to see the size if we want that information. ''' 
# data_loader_iterator = iter(loaders['train'])
# first_batch = next(data_loader_iterator)
# print(first_batch.shape)



In [16]:
@torch.jit.script
def split_batches_of_minute_segments(batch):
    num_channels = 18
    chunks = torch.split(batch, 1, 0)
    list0 = [torch.stack(torch.split(chunk.view(num_channels, 3600), 60, 1), dim=0) for chunk in chunks]
    new_batch = torch.cat(list0,dim=0)
    # print('new_batch',new_batch.shape)
    return new_batch

@torch.jit.script
def merge_seconds_to_minute(encoded):
    # batch_size = 16
    # print(encoded.shape)
    # chunks = torch.split(encoded, 1, 0)
    # print('chunks[0].shape',chunks[0].shape)
    # print('len(chunks)',len(chunks))
    # list = [torch.stack(chunks[i*60: i*60 + 60]) for i in range(batch_size)]
    # print('list[0].shape',list[0].shape)
    # encoded_batch = torch.cat(list, dim=0)
    # print('encoded_batch.shape',encoded_batch.shape)
    chunks = torch.split(encoded,1,0)
    list0 = []
    # print(chunks[0].shape)
    for chunk in chunks:
        chunk = chunk.view(chunk.size(1), chunk.size(2))
        # print(chunk.shape)
        list0.append(chunk)
    list1=[]
    # print(len(list0))
    # print(list0[0].size)
    for i in range(16):
        list1.append(torch.cat(list0[i*60:i*60+60],dim=1))
    # print(len(list1))
    # print(list1[0].shape)
    final = torch.stack(list1)    
    # print(final.shape)
    return final
    
    

In [17]:

def train_model_with_virtual_epochs(model, criterion, optimizer, loaders, device, encoder, num_epochs=1):
    # print(0)
    scaler = GradScaler()
    best_loss = float('inf')
    best_model = None
    
    vepoch_total = 0
    vepoch_loss_sum = 0
    best_vepoch_loss = float('inf')
    early_stopping_patience = 0
    encoder.eval()
    # print(1)
    for epoch in range(num_epochs):
        model.train()
        train_loader_tqdm = tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
        virtual_epoch_start_time = time.time()

        # Initialize variables for tracking gradient and parameter stats
        grad_max = float('-inf')
        grad_min = float('inf')
        param_max = float('-inf')
        param_min = float('inf')
        # print(2)
        for inputs_cpu, labels in train_loader_tqdm:
            optimizer.zero_grad()
            batch_to_encode = split_batches_of_minute_segments(inputs_cpu)
            batch_to_encode_gpu = batch_to_encode.to(device)
            encoded_gpu = encoder(batch_to_encode_gpu)
            encoded_inputs_gpu = merge_seconds_to_minute(encoded_gpu)

            
            
            # optimizer.zero_grad()
            # print(encoded_inputs_gpu.shape)
            output_gpu = model(encoded_inputs_gpu)
            # output_gpu = model(inputs_cpu.to(device))
            # print(output_gpu.shape)
            loss = criterion(output_gpu, labels.to(device)) 
            # print(output_gpu)
            # print(labels)
            # print(loss)
            loss.backward()
            # scaler.scale(loss).backward()

            # Track max and min of gradients
            batch_grad_max = max((p.grad.max().item() for p in model.parameters() if p.grad is not None), default=grad_max)
            batch_grad_min = min((p.grad.min().item() for p in model.parameters() if p.grad is not None), default=grad_min)
            grad_max = max(grad_max, batch_grad_max)
            grad_min = min(grad_min, batch_grad_min)

            clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            # scaler.step(optimizer)
            # scaler.update()

            vepoch_total += 1
            vepoch_loss_sum += loss.item()

            if time.time() - virtual_epoch_start_time > 15:
                vepoch_loss = vepoch_loss_sum / vepoch_total
                if best_vepoch_loss > vepoch_loss:
                    best_vepoch_loss = vepoch_loss
                else:
                    early_stopping_patience += 1

                # Calculate max and min of model parameters at the end of the virtual epoch
                param_max = max(p.data.max().item() for p in model.parameters())
                param_min = min(p.data.min().item() for p in model.parameters())

                train_loader_tqdm.set_postfix(
                    Best=f'{best_vepoch_loss:.10f}',
                    Vepoch=f'{vepoch_loss:.10f}',
                    patience=early_stopping_patience,
                    Grad_Max=grad_max,
                    Grad_Min=grad_min,
                    Param_Max=param_max,
                    Param_Min=param_min
                )
                # print('Grad Max:', grad_max, ' Grad Min:', grad_min)
                virtual_epoch_start_time = time.time()
                vepoch_total = 0
                vepoch_loss_sum = 0
                grad_max = float('-inf')  # Reset for next virtual epoch
                grad_min = float('inf')   # Reset for next virtual epoch
            # break

    return best_model

In [18]:

from Encode_ResNet_Model import ResNet50, ResNet101
from Simple_Test_Model import Model
from autoencoder_classify.Convolutional_Encoder_Model import ResNet_Encoder

channels = 18
batch_size = 16
num_classes = 5
# model = ResNet50(num_classes=5, channels=15).to('cuda')

model = Model()
# model = torch.compile(model, mode = 'default')
# model = torch.compile(model,mode = 'max-autotune')
model.to('cuda')


channels = 9 + 5 + 4
# Build model
encoder = ResNet_Encoder(channels)
encoder.load_state_dict(torch.load('../../melee_project_data/convolutional_autoencode_bottlenecksize_15_weights_2.pt'))
encoder.to('cuda')

# Set the model to evaluation mode

# Optimize the loaded model with Torch-TensorRT
# encoder = torch_tensorrt.compile(
#     encoder,
#     inputs=[torch_tensorrt.Input((batch_size * 60, channels, 60))],
#     enabled_precisions={torch.float}  # Use torch.half for FP16 precision if needed
# )
## Optionally compile the model
# import torch_tensorrt
# model = torch.compile(model, mode = 'default')
# model = torch.compile(model,mode = 'max-autotune')


# encoder = torch.compile(encoder, backend="torch_tensorrt")
# model = torch.compile(model, backend="torch_tensorrt",mode = 'max-autotune')

# Move the optimized model to GPU
# trt_encoder = trt_encoder.cuda()


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Autoencoder(
  (encoder): Encoder(
    (layer1): Sequential(
      (0): Encoder_Bottleneck(
        (conv1): Conv1d(18, 64, kernel_size=(1,), stride=(1,))
        (batch_norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (batch_norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,))
        (batch_norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (i_downsample): Sequential(
          (0): Conv1d(18, 256, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU()
      )
      (1): Encoder_Bottleneck(
        (conv1): Conv1d(256, 64, kernel_size=(1,), stride=(1,))
        (batch_norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, 

In [19]:
# torch.autograd.set_detect_anomaly(False)
# Pepare data loaders
batch_size =  16
num_workers = 16
# print(1)
loaders = prepare_data_loaders(train_df, test_df, batch_size, num_workers)
# print(2)
optimizer = Adam(model.parameters(), lr=0.0005)
# print(3)
criterion = nn.CrossEntropyLoss() # All our batches will be the same.

# print(4)
num_epochs = 1

# def initialize_parameters(model):
#     for m in model.modules():
#         if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#             torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
#             if m.bias is not None:
#                 torch.nn.init.constant_(m.bias, 0)

# # Initialize model parameters
# initialize_parameters(model)


gc.collect()
torch.cuda.empty_cache()
# print(5)
train_model_with_virtual_epochs(model, criterion, optimizer, loaders, 'cuda', encoder, num_epochs)

Epoch 1/1:   4%|▎         | 1196/33917 [01:27<40:02, 13.62batch/s, Best=1.4047853617, Grad_Max=0.848, Grad_Min=-0.864, Param_Max=0.185, Param_Min=-0.234, Vepoch=1.4047853617, patience=0]


KeyboardInterrupt: 

In [ ]:
# asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)


In [ ]:
x = [3, 4, 4]
plus = 3 + 2
multiplier = 4
for val in x:
    plus += multiplier * val
    multiplier *= 2
print(1 + 2 * plus)
    
# print(1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32))

227


In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)


Epoch 1/2:   0%|          | 0/27133 [00:00<?, ?batch/s]

Epoch 2/2:  61%|██████▏   | 16649/27133 [01:01<00:38, 271.70batch/s, accuracy=82.2, loss=0.03]  


KeyboardInterrupt: 

In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)

In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)

In [ ]:
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)

In [ ]:
array = np.array([[0,0.1,0.2],[1,1.1,1.2], [2,2.1,2.2],[3,3.1,3.2]])
array

array([[0. , 0.1, 0.2],
       [1. , 1.1, 1.2],
       [2. , 2.1, 2.2],
       [3. , 3.1, 3.2]])

In [ ]:
np.reshape(array,(3,3,1))

array([[[0. ],
        [0.1],
        [0.2]],

       [[1. ],
        [1.1],
        [1.2]],

       [[2. ],
        [2.1],
        [2.2]]])

In [ ]:
array = np.array([[0,0.1,0.2],[1,1.1,1.2], [2,2.1,2.2], [3,3.1,3.2]])
# array = # a 14x3600 arary
new_array = np.zeros((3,4,1))
for i in range(3):
    segment = array[:, i]
    segment = np.reshape(segment,(-1,1))
    # print(segment.shape)
    new_array[i] += segment
    # print(segment)
print(new_array)

tensor = torch.from_numpy(array)
tensor = tensor.view(3,1,4)
torch.transpose(tensor,2,0)


AttributeError: module 'numpy' has no attribute 'arrange'

In [ ]:
tensor = torch.from_numpy(array).transpose(0, 1)
# Assuming the original shape is (4, 3), transposing will make it (3, 4)
# Then reshape it to (3, 4, 1) to match your numpy operation
reshaped_tensor = tensor.view(3, 4, 1)
print(reshaped_tensor)

tensor([[[0.0000],
         [1.0000],
         [2.0000],
         [3.0000]],

        [[0.1000],
         [1.1000],
         [2.1000],
         [3.1000]],

        [[0.2000],
         [1.2000],
         [2.2000],
         [3.2000]]], dtype=torch.float64)


In [ ]:
a = np.arange(10)
a = a.reshape(1,-1)
a = np.repeat(a,3,0)
print(a)
print(a.shape)

[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
(3, 10)


In [ ]:
print(a)
transformed_a = np.array([a[:,i*2:i*2+2] for i in range(5)])
print(transformed_a)

[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
[[[0 1]
  [0 1]
  [0 1]]

 [[2 3]
  [2 3]
  [2 3]]

 [[4 5]
  [4 5]
  [4 5]]

 [[6 7]
  [6 7]
  [6 7]]

 [[8 9]
  [8 9]
  [8 9]]]


In [ ]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

# Define the original array
a = np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

# Define parameters for the transformation
n_rows, n_cols = a.shape
window_width = 2
n_windows = n_cols // window_width

# Create a view with as_strided
new_shape = (n_windows, n_rows, window_width)
new_strides = (a.strides[1] * window_width, a.strides[0], a.strides[1])

transformed_a = as_strided(a, shape=new_shape, strides=new_strides)

print(transformed_a)


[[[0 1]
  [0 1]
  [0 1]]

 [[2 3]
  [2 3]
  [2 3]]

 [[4 5]
  [4 5]
  [4 5]]

 [[6 7]
  [6 7]
  [6 7]]

 [[8 9]
  [8 9]
  [8 9]]]


In [ ]:
b = torch.Tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
print(b)


tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])


In [ ]:
# rewrite this using list comprehension for the loops
import torch

# Define the original tensor
for _ in range(100000):
    b = torch.Tensor([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])
    # Add 0.1 to the second row of each batch
    b[1, :, :] += 0.1

    # print(b)
    chunks = torch.split(b,1,0)
    # print(chunks[0].view(3,10).shape)

    # print(chunks)
    list = []
    for chunk in chunks:
        # print(chunk.shape)
        list.append(torch.stack(torch.split(chunk.view(3,10),2,1),dim=0))

    # print(list)
    # print(list[0].shape)
    new = torch.cat(list,dim=0)
    # print(new.shape)
    # print(new)

    batch_chunks = torch.split(new,1,0)
    # print(batch_chunks[0:1])
    list = []
    for i in range(2):
        list.append(torch.stack(batch_chunks[i*5:i*5 + 5]))
    # print(list)
    final = torch.cat(list,dim=0)
print(final)
# batch = torch.stack(chunks, dim=0)
# print(batch)
# batch_chunks = torch.split(batch,1,0)
# print(batch_chunks)
# original_b = torch.cat(batch_chunks,dim=2)
# print(original_b)

tensor([[[[0.0000, 1.0000],
          [0.0000, 1.0000],
          [0.0000, 1.0000]]],


        [[[2.0000, 3.0000],
          [2.0000, 3.0000],
          [2.0000, 3.0000]]],


        [[[4.0000, 5.0000],
          [4.0000, 5.0000],
          [4.0000, 5.0000]]],


        [[[6.0000, 7.0000],
          [6.0000, 7.0000],
          [6.0000, 7.0000]]],


        [[[8.0000, 9.0000],
          [8.0000, 9.0000],
          [8.0000, 9.0000]]],


        [[[0.1000, 1.1000],
          [0.1000, 1.1000],
          [0.1000, 1.1000]]],


        [[[2.1000, 3.1000],
          [2.1000, 3.1000],
          [2.1000, 3.1000]]],


        [[[4.1000, 5.1000],
          [4.1000, 5.1000],
          [4.1000, 5.1000]]],


        [[[6.1000, 7.1000],
          [6.1000, 7.1000],
          [6.1000, 7.1000]]],


        [[[8.1000, 9.1000],
          [8.1000, 9.1000],
          [8.1000, 9.1000]]]])


In [ ]:

b = torch.Tensor([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])
# Add 0.1 to the second row of each batch
b[1, :, :] += 0.1
b[2, :, :] += 0.2

# print(b)
chunks = torch.split(b,1,0)
# print(chunks[0].view(3,10).shape)

# print(chunks)
list = []
for chunk in chunks:
    # print(chunk.shape)
    list.append(torch.stack(torch.split(chunk.view(3,10),2,1),dim=0))

# print(list[0])
# print(list[1])
# print(list[2])
# print(list[0].shape)
new = torch.cat(list,dim=0)
# print(new.shape)
# print(new)

chunks = torch.split(new,1,0)
# print(len(chunks))
# print(chunks[0].shape)
# print(chunks[5])
list = []
for chunk in chunks:
    list.append(chunk.view(3,2))
# print(len(list))
# print(list[0].shape)
# print(list[0])
list1=[]
for i in range(3):
    list1.append(torch.cat(list[i*5:i*5+5],dim=1))
# print(list1[0].shape)
final = torch.stack(list1)
print(final)

tensor([[[0.0000, 1.0000, 2.0000, 3.0000, 4.0000, 5.0000, 6.0000, 7.0000,
          8.0000, 9.0000],
         [0.0000, 1.0000, 2.0000, 3.0000, 4.0000, 5.0000, 6.0000, 7.0000,
          8.0000, 9.0000],
         [0.0000, 1.0000, 2.0000, 3.0000, 4.0000, 5.0000, 6.0000, 7.0000,
          8.0000, 9.0000]],

        [[0.1000, 1.1000, 2.1000, 3.1000, 4.1000, 5.1000, 6.1000, 7.1000,
          8.1000, 9.1000],
         [0.1000, 1.1000, 2.1000, 3.1000, 4.1000, 5.1000, 6.1000, 7.1000,
          8.1000, 9.1000],
         [0.1000, 1.1000, 2.1000, 3.1000, 4.1000, 5.1000, 6.1000, 7.1000,
          8.1000, 9.1000]],

        [[0.2000, 1.2000, 2.2000, 3.2000, 4.2000, 5.2000, 6.2000, 7.2000,
          8.2000, 9.2000],
         [0.2000, 1.2000, 2.2000, 3.2000, 4.2000, 5.2000, 6.2000, 7.2000,
          8.2000, 9.2000],
         [0.2000, 1.2000, 2.2000, 3.2000, 4.2000, 5.2000, 6.2000, 7.2000,
          8.2000, 9.2000]]])


In [ ]:
b = torch.Tensor([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])
# Add 0.1 to the second row of each batch
b[1, :, :] += 0.1
b[2, :, :] += 0.2

print(b.view(-1, 3, 3, 2))




tensor([[[[0.0000, 1.0000],
          [2.0000, 3.0000],
          [4.0000, 5.0000]],

         [[6.0000, 7.0000],
          [8.0000, 9.0000],
          [0.0000, 1.0000]],

         [[2.0000, 3.0000],
          [4.0000, 5.0000],
          [6.0000, 7.0000]]],


        [[[8.0000, 9.0000],
          [0.0000, 1.0000],
          [2.0000, 3.0000]],

         [[4.0000, 5.0000],
          [6.0000, 7.0000],
          [8.0000, 9.0000]],

         [[0.1000, 1.1000],
          [2.1000, 3.1000],
          [4.1000, 5.1000]]],


        [[[6.1000, 7.1000],
          [8.1000, 9.1000],
          [0.1000, 1.1000]],

         [[2.1000, 3.1000],
          [4.1000, 5.1000],
          [6.1000, 7.1000]],

         [[8.1000, 9.1000],
          [0.1000, 1.1000],
          [2.1000, 3.1000]]],


        [[[4.1000, 5.1000],
          [6.1000, 7.1000],
          [8.1000, 9.1000]],

         [[0.2000, 1.2000],
          [2.2000, 3.2000],
          [4.2000, 5.2000]],

         [[6.2000, 7.2000],
          [8.2000, 9

In [ ]:
import torch
import time

# Define the function to be compiled with TorchScript
@torch.jit.script
def manipulate_tensor(b):

    # Add 0.1 to the second row of each batch
    b[:, 1, :] += 0.1

    # Split tensor into chunks of size 1 along the first dimension
    chunks = torch.split(b, 1, 0)

    # Stack and reshape chunks into the desired format
    list1 = [torch.stack(torch.split(chunk.view(3, 10), 2, 1), dim=0) for chunk in chunks]

    # Concatenate the list into a single tensor
    new = torch.cat(list1, dim=0)

    # Split the concatenated tensor into chunks
    batch_chunks = torch.split(new, 1, 0)

    # Create the final list of stacked tensors
    list2 = [torch.stack([batch_chunks[i*5 + j].view(3,2) for j in range(5)]) for i in range(2)]

    # Concatenate the final list into a single tensor
    final = torch.cat(list2, dim=0)
    
    return final
b = torch.Tensor([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])

# Call the compiled function
final = manipulate_tensor(b)
# Main loop with timing
start_time = time.time()

for _ in range(100000):
    # Call the compiled function
    final = manipulate_tensor(b)

end_time = time.time()
print(f"TorchScript time: {end_time - start_time:.6f} seconds")

TorchScript time: 2.962586 seconds


In [ ]:
import torch
import time

# Define the function to be compiled with TorchScript
@torch.jit.script
def manipulate_tensor(b):

    # Add 0.1 to the second row of each batch
    b[1, :, :] += 0.1

    # print(b)
    chunks = torch.split(b,1,0)
    # print(chunks[0].view(3,10).shape)

    # print(chunks)
    list = []
    for chunk in chunks:
        # print(chunk.shape)
        list.append(torch.stack(torch.split(chunk.view(3,10),2,1),dim=0))

    # print(list)
    # print(list[0].shape)
    new = torch.cat(list,dim=0)
    print(new.shape)
    # print(new)

    batch_chunks = torch.split(new,1,0)
    # print(batch_chunks[0:1])
    list = []
    for i in range(2):
        list.append(torch.stack(batch_chunks[i*5:i*5 + 5]))
    # print(list)
    final = torch.cat(list,dim=0)
    
    return final
b = torch.Tensor([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
                    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]]).to('cuda')

# Call the compiled function
final = manipulate_tensor(b)
print(final)
# Main loop with timing
# start_time = time.time()

# for _ in range(100000):
#     # Call the compiled function
#     final = manipulate_tensor(b)

# end_time = time.time()
# print(f"TorchScript time: {end_time - start_time:.6f} seconds")


[10, 3, 2]
tensor([[[[0.0000, 1.0000],
          [0.0000, 1.0000],
          [0.0000, 1.0000]]],


        [[[2.0000, 3.0000],
          [2.0000, 3.0000],
          [2.0000, 3.0000]]],


        [[[4.0000, 5.0000],
          [4.0000, 5.0000],
          [4.0000, 5.0000]]],


        [[[6.0000, 7.0000],
          [6.0000, 7.0000],
          [6.0000, 7.0000]]],


        [[[8.0000, 9.0000],
          [8.0000, 9.0000],
          [8.0000, 9.0000]]],


        [[[0.1000, 1.1000],
          [0.1000, 1.1000],
          [0.1000, 1.1000]]],


        [[[2.1000, 3.1000],
          [2.1000, 3.1000],
          [2.1000, 3.1000]]],


        [[[4.1000, 5.1000],
          [4.1000, 5.1000],
          [4.1000, 5.1000]]],


        [[[6.1000, 7.1000],
          [6.1000, 7.1000],
          [6.1000, 7.1000]]],


        [[[8.1000, 9.1000],
          [8.1000, 9.1000],
          [8.1000, 9.1000]]]], device='cuda:0')


In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, encoder, resnet):
        super(Classifier, self).__init__()
        self.encoder = encoder  # Assuming encoder is a pre-trained model accepting (batch_size, 18, 60)
        self.resnet = resnet    # CNN model that will process the encoded output

    def forward(self, x):
        # x shape: (batch_size, 18, 3600)
        # Reshape x to (batch_size, 60, 18, 60) where each 18x60 block is 1 second of data
        x = torch.split(x,60,1)

        x = self.encoder(x)  # Expecting encoder output of shape (batch_size * 60, 15, 4)
        
        x = torch.cat(x,1)
        

        # Pass through ResNet or another CNN
        x = self.resnet(x)  # Ensure that self.resnet is designed to take this shape
        return x